### Скачиваем необходимые библиотеки

In [21]:
!pip install requests
!pip install beautifulsoup4

### Импорты

In [135]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep

## Книгопоиск

### Пример для одной книги на странице

#### Подготовка

Берем ссылку на первую страницу, с которой хотим парсить информацию, помещаем в переменную url.

In [26]:
url = "https://knigopoisk.org/bookslist/200_luchshih_knig_po_versii_BBC"

Прописываем переменную **headers**, чтобы имитировать браузер.

Есть вероятность, что получим мобильную версию сайта или версию сайта для ботов. Дабы избежать этого, нужно притвориться браузером. Для этого нужно зименить заголовки на заголовки браузеров Chrome/Mozilla и т.д. при парсинге.

Также с помощью **timeout** снизим скорость запросов.

In [29]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36",
    "Content-Type":"text"
}

timeout=30

Отправляем запрос на сервер с помощью метода **.get()**, передав необходимые параметры.

In [30]:
r = requests.get(url, timeout=timeout, headers=headers)

Проверяем содержимое ответа с помощью атрибута **.text**.

In [ ]:
# r.text

Создаем объект класса BeautifulSoup, в качестве параметров передаем полученный текст, и указываем формат, в котором нам должен будет вернуться ответ.

In [32]:
soup = BeautifulSoup(r.text, 'lxml')

Проверяем содержимое этого объекта. Получаем код страницы.

In [ ]:
# soup

#### Работа с объектом BeautifulSoup

Будем парсить книги, элемент книги -

```
<div class="main-list-item block-table" id="2290">

    <div class="main-list-item__left block-table__cell"><img itemprop="image" src="/media/books/Dz/55x83/Dzh_55x83.jpg" width="55" alt="Гарри Поттер и философский камень">
        
    </div>
    <div class="main-list-item__right block-table__cell">

        <div class="main-list-item-right-1 block-table">
            <div class="main-list-item-right-1__left block-table__cell">
                <div><a href="https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen" title="Гарри Поттер и философский камень" class="list-book__link">Гарри Поттер и философский камень</a></div>
                <div class="list-book__link-author"><a class="list-author" href="/authors/dzh_k_rouling" title="Дж. К. Роулинг">Дж. К. Роулинг</a></div>
            </div>
                    </div>
        <div class="main-list-item-right-3 block-table">
            <div class="main-list-item-right-3__left block-table__cell">
                <div class="block-table">
                    <div class="block-table__cell rating-list-block-stars-container">

                                                            <img src="/img/rating-guest-9.png" alt="Рейтинг: 8.99">
                            
                    </div>
                    <div class="block-table__cell pdl10">
                        <span class="rating">8.99</span><span class="count-rating">(1903)</span>
                    </div>
                </div>
            </div>
            <div class="main-list-item-right-3__right block-table__cell">
                                
                                
                                    <a href="https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen/citaty" class="count-quotes" title="Цитаты из книги «Гарри Поттер и философский камень»"><span class="display-swither">Цитаты (</span><span>27</span><span class="display-swither">)</span></a>
                                            </div>
        </div>
                    <div class="main-list-item-right-description">Издательская группа "Азбука-Аттикус" с гордостью сообщает о начале публикации уникальных мировых бестселлеров - семи книг Дж.К. Роулинг о Гарри Поттере. Самая знаменитая сага новейшего времени разошлась рекордным тиражом - более 400 миллионов экземпляров на 68 языках (включая эсперанто, древнегреческий и латынь). Книги Роулинг сумели оторвать детей и взрослых от экранов компьютеров и стали мощнейшим импульсом для интереса к чтению в современную эпоху. Приобретение исключительных прав на издание ...</div>            </div>

  </div>
```



Найдем 1-ую книгу на странице (её тег `div`, а класс `main-list-item block-table`) с помощью метода **.find()**, возращающего 1-ое вхождение.

In [34]:
soup.find('div', class_='main-list-item block-table')

<div class="main-list-item block-table" id="2290">
<div class="main-list-item__left block-table__cell"><img alt="Гарри Поттер и философский камень" itemprop="image" src="/media/books/Dz/55x83/Dzh_55x83.jpg" width="55"/>
</div>
<div class="main-list-item__right block-table__cell">
<div class="main-list-item-right-1 block-table">
<div class="main-list-item-right-1__left block-table__cell">
<div><a class="list-book__link" href="https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen" title="Гарри Поттер и философский камень">Гарри Поттер и философский камень</a></div>
<div class="list-book__link-author"><a class="list-author" href="/authors/dzh_k_rouling" title="Дж. К. Роулинг">Дж. К. Роулинг</a></div>
</div>
</div>
<div class="main-list-item-right-3 block-table">
<div class="main-list-item-right-3__left block-table__cell">
<div class="block-table">
<div class="block-table__cell rating-list-block-stars-container">
<img alt="Рейтинг: 8.99" src="/img/rating-guest-9.png"/

Углубимся дальше - достанем название книги и ссылку.

* (тег - div, класс - main-list-item block-table),
* (тег - div, класс - main-list-item__right block-table__cell),
* (тег - div, класс - main-list-item-right-1 block-table),
* (тег - div, класс - main-list-item-right-1__left block-table__cell),
* (тег - a, класс - list-book__link)


In [74]:
title_and_link = soup.find('div', class_='main-list-item block-table') \
  .find('div', class_='main-list-item__right block-table__cell') \
  .find('div', class_='main-list-item-right-1 block-table') \
  .find('div', class_='main-list-item-right-1__left block-table__cell') \
  .find('a', class_='list-book__link')

title_and_link

<a class="list-book__link" href="https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen" title="Гарри Поттер и философский камень">Гарри Поттер и философский камень</a>

Но не нужно обязательно проходить по всему пути тег а с таким классом один.

In [75]:
title_and_link = soup.find('a', class_='list-book__link')

title_and_link

<a class="list-book__link" href="https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen" title="Гарри Поттер и философский камень">Гарри Поттер и философский камень</a>

Видим, что в атрибутах хранится ссылка и название книги, также название книги находится внутри тега. Добудем ссылку с помощью метода **.get()**, передав ему название атрибута тега. А название получим через атрибут **.text**.

In [46]:
link = title_and_link.get('href')
title = title_and_link.text

print(link)
print(title)

https://knigopoisk.org/books/dzh_k_rouling_garri_potter_i_filosofskiy_kamen
Гарри Поттер и философский камень


Углубимся дальше - достанем имя автора.

* (тег - div, класс - main-list-item block-table),
* (тег - div, класс - main-list-item__right block-table__cell),
* (тег - div, класс - main-list-item-right-1 block-table),
* (тег - div, класс - main-list-item-right-1__left block-table__cell),
* (тег - div, класс - list-book__link-author)
* (тег - a, класс - list-author)

Но не нужно обязательно проходить по всему пути тег а с таким классом один.

In [73]:
author = soup.find('a', class_='list-author').get('title')

author

'Дж. К. Роулинг'

Углубимся дальше - достанем рейтинг.

* (тег - div, класс - main-list-item block-table),
* (тег - div, класс - main-list-item__right block-table__cell),
* (тег - div, класс - block-table__cell rating-list-block-stars-container),
* (тег - img)

Ещё понадобились некоторые предобразования, чтобы получить то, что нужно.

In [76]:
rating = soup.find('div', class_='main-list-item block-table') \
  .find('div', class_='main-list-item__right block-table__cell') \
  .find('div', class_='block-table__cell rating-list-block-stars-container') \
  .find('img').get('alt').split()[1]

rating = float(rating)
rating

8.99

**!!!** Если вдруг сочетание *тег+класс* встречается не один раз, а нам нужен определенный из этих блоков, то нужно использовать метод **.findAll()**, возвращающий список. В этом списке по порядку можно выбрать то, что нужно.

### Пример для одной страницы

Соберем все книги в один список.

In [89]:
books = soup.findAll('div', class_='main-list-item block-table')
len(books)

155

Соберем всю информацию о книгах с помощью цикла в предеменную data.

In [90]:
data = []

Так как данные иногда могут отсутствовать, то нужно использовать **try-except**, для примера, в этот блок обернуто название со ссылкой -  по логике такая книга не будет обрабатываться.

In [91]:
for book in books:
  try:
    title_and_link = book.find('a', class_='list-book__link')
    link = title_and_link.get('href')
    title = title_and_link.text
  except:
    continue

  author = book.find('a', class_='list-author').get('title')

  rating = book.find('div', class_='main-list-item__right block-table__cell') \
  .find('div', class_='block-table__cell rating-list-block-stars-container') \
  .find('img').get('alt').split()[1]
  rating = float(rating)

  data.append([title, author, link, rating])

In [92]:
len(data)

152

Всю эту информацию можно собрать в датафрейм и при необходимости перевести в csv-файл.

In [93]:
table_head = ['title', 'author', 'link', 'rating']

In [94]:
df = pd.DataFrame(data, columns=table_head)

In [95]:
df

,title,author,link,rating
0,Гарри Поттер и философский камень,Дж. К. Роулинг,https://knigopoisk.org/books/dzh_k_rouling_gar...,8.99
1,Гарри Поттер и узник Азкабана,Дж. К. Роулинг,https://knigopoisk.org/books/dzh_k_rouling_gar...,8.98
2,Гарри Поттер и Кубок Огня,Дж. К. Роулинг,https://knigopoisk.org/books/dzh_k_rouling_gar...,8.98
3,Унесенные ветром,Маргарет Митчелл,https://knigopoisk.org/books/margaret_mitchell...,8.97
4,Граф Монте-Кристо,Александр Дюма,https://knigopoisk.org/books/aleksandr_dyuma_g...,8.97
...,...,...,...,...
147,Осиная фабрика,Иэн Бэнкс,https://knigopoisk.org/books/ien_benks_osinaya...,6.06
148,Американский психопат,Брет Истон Эллис,https://knigopoisk.org/books/bret_iston_ellis_...,6.05
149,В дороге,Джек Керуак,https://knigopoisk.org/books/dzhek_keruak_v_do...,6.03
150,Алхимик,Пауло Коэльо,https://knigopoisk.org/books/paulo_koelo_alkhimik,6.01


### Пример для нескольких страниц с пролистыванием

На сайте книгопоиска есть страницы с цитатами, их можно пролистывать, поэтому их будем использовать для парсинга.

#### Подготовка

In [117]:
url = "https://knigopoisk.org/quote/index?page=1"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36",
    "Content-Type":"text"
}

timeout=30
r = requests.get(url, timeout=timeout, headers=headers)
soup = BeautifulSoup(r.text, 'lxml')
# soup

#### Работа с объектом BeautifulSoup

Добудем 1 цитату.

```
<div class="quotesbook">
    <p class="quote-head">Цитата из книги «Над пропастью во ржи»</p>
    <div class="quote" id="remove_quote">
        <p>«"Тело женщины - скрипка и что надо быть прекрасным музыкантом, чтобы заставить его звучать."</p><p>»</p>
                <a href="https://knigopoisk.org/books/dzherom_d_selindzher_nad_propastyu_vo_rzhi/citaty" class="all-quotes-book">Все цитаты из этой книги</a>
    </div>

                <div class="likebox dislike" id="quote-11985">

        <a href="/site/login" rel="nofollow" title="Добавить в любимые">Мне нравится</a>
    
    <p class="count-like">  <span id="req_res11985">0</span></p>
</div>    
</div>
```



In [118]:
q = soup.find('div', class_='quotesbook')
q

<div class="quotesbook">
<p class="quote-head">Цитата из книги «Над пропастью во ржи»</p>
<div class="quote" id="remove_quote">
<p>«"Тело женщины - скрипка и что надо быть прекрасным музыкантом, чтобы заставить его звучать."</p><p>»</p>
<a class="all-quotes-book" href="https://knigopoisk.org/books/dzherom_d_selindzher_nad_propastyu_vo_rzhi/citaty">Все цитаты из этой книги</a>
</div>
<div class="likebox dislike" id="quote-11985">
<a href="/site/login" rel="nofollow" title="Добавить в любимые">Мне нравится</a>
<p class="count-like"> <span id="req_res11985">0</span></p>
</div>
</div>

Добудем название книги и саму цитату.

In [119]:
q.findAll('p')

[<p class="quote-head">Цитата из книги «Над пропастью во ржи»</p>,
 <p>«"Тело женщины - скрипка и что надо быть прекрасным музыкантом, чтобы заставить его звучать."</p>,
 <p>»</p>,
 <p class="count-like"> <span id="req_res11985">0</span></p>]

In [120]:
book = q.findAll('p')[0].text[len('Цитата из книги «'):-1]
book

'Над пропастью во ржи'

In [141]:
text = q.findAll('p')[1].text[1:]
text

'"Тело женщины - скрипка и что надо быть прекрасным музыкантом, чтобы заставить его звучать."'

#### Работа с одной страницей

In [122]:
quotes = soup.findAll('div', class_='quotesbook')
len(quotes)

20

In [130]:
data = []

In [131]:
for quote in quotes:
  try:
    book_and_text = quote.findAll('p')
  except:
    continue

  book = book_and_text[0].text[len('Цитата из книги «'):-1]
  text = book_and_text[1].text[2:-1]

  data.append([book, text])

In [132]:
len(data)

20

In [ ]:
# data

#### Работа с несколькими страницами

In [150]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36",
    "Content-Type":"text"
}

timeout=30

data = []

In [155]:
for page in range(1,11):
  url = f"https://knigopoisk.org/quote/index?page={page}"
  r = requests.get(url, timeout=timeout, headers=headers)
  sleep(3)
  soup = BeautifulSoup(r.text, 'lxml')

  quotes = soup.findAll('div', class_='quotesbook')

  for quote in quotes:
    try:
      book_and_text = quote.findAll('p')
    except:
      continue

    book = book_and_text[0].text[len('Цитата из книги «'):-1]
    text = book_and_text[1].text[1:-1]

    data.append([book, text])

  print(page)


1
2
3
4
5
6
7
8
9
10


In [156]:
len(data)

400

In [157]:
table_head = ['book', 'text']
df = pd.DataFrame(data, columns=table_head)

In [161]:
df

,book,text
0,Над пропастью во ржи,"""Тело женщины - скрипка и что надо быть прекра..."
1,Убить пересмешника…,"""Человека по-настоящему узнаешь только тогда, ..."
2,1984,"""Они подобны муравью, который видит мелкое и н..."
3,Куриный бульон для души: 101 лучшая история,"Жизнь всегда налаживается, когда вы начинаете ..."
4,Охота на царя,…государь попал под ее влияние... Он устал от ...
...,...,...
395,Госпожа Бовари,"А между тем она была полна вожделений, яростны..."
396,Госпожа Бовари,"Он любил её чистой любовью, - такая любовь не ..."
397,Госпожа Бовари,"Я со всеми дамами лажу, кроме собственной жены!"
398,Госпожа Бовари,"Ничего, однако, достопримечательного не было в..."
